In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "drive/My Drive/Colab Notebooks/SIH"

/content/drive/My Drive/Colab Notebooks/SIH


In [ ]:
!pip install tf-nightly

     |████████████████████████████████| 325.2MB 44kB/s 
     |████████████████████████████████| 460kB 44.0MB/s 
     |████████████████████████████████| 6.7MB 50.8MB/s 


In [ ]:
!pip install efficientnet==1.1.0

In [ ]:
import os
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras as tfkeras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Input
from tensorflow.keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger
from tensorflow.keras.callbacks import ReduceLROnPlateau,LearningRateScheduler
from efficientnet.tfkeras import EfficientNetB7, EfficientNetB0, EfficientNetB4

class parameters:
    imgsize = 380
    depth = 3
    BATCH_SIZE = 4
    ILR = 1e-2
    FLR = 1e-3
    val_batch_size = 4
    initial_epoch = 1
    final_epoch = 0
    total_epochs = initial_epoch + final_epoch
    
def create_model(height, width, depth):

    # initialize the input shape
    inputShape = (height, width, depth)
    #extract features from pretrained model mobile-net
    base_model = EfficientNetB4(weights='imagenet',
                include_top=False, input_shape=inputShape)


    return base_model

def effn7(base_model, classes):

    x = base_model.output
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    preds = Dense(classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)  

    return model  

# def effnb7_finetune(base_model, classes):
#     x = base_model.output
#     # define classification layers and increase learnable parameters
#     x = Dense(1024, activation="relu")(x)
#     x = BatchNormalization()(x)
#     x = GlobalAveragePooling2D()(x)
#     x = BatchNormalization()(x)
#     preds = Dense(classes, activation='softmax')(x)
#     model = Model(inputs=base_model.input, outputs=preds)

#     return model


def train(epoch, model, classes):
  
    
    print("[LOGGING] training initial network...")
    for layer in model.layers:
        layer.trainable = False
    model = effn7(model, classes)
    # initialize the optimizer compile the model and
    opt = Adam(lr=parameters.ILR)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])    
    # print(model.summary())
    # train the network
    model_dir = 'ValAccTransfer.h5'

    # construct the set of callbacks
    callbacks = [ModelCheckpoint(filepath=model_dir, verbose=1, save_best_only=True, monitor='val_accuracy'),
                 ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1)]

    _ = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // parameters.BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // parameters.val_batch_size,
        epochs=parameters.initial_epoch,
        workers=0,
        callbacks=callbacks,
        verbose=1)
    
    model.save("FinalTransfer.h5")
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_model_name = "FinalTransfer.tflite"
    open(tflite_model_name, "wb").write(tflite_model)
    print("[INFO] Fine tuning and continue train")
    print("[LOGGING] training final network...")

Using TensorFlow backend.


In [ ]:
train_image_gen = ImageDataGenerator(
        rescale=1.0/255,
        zca_whitening=False,  
        rotation_range=15,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=False)
 
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_image_gen.flow_from_directory("data1/train",
                                                      target_size=(parameters.imgsize, parameters.imgsize),
                                                      batch_size=parameters.BATCH_SIZE,
                                                      class_mode='categorical') # set as training data

validation_generator = test_datagen.flow_from_directory("data1/val",
                                                        target_size=(parameters.imgsize, parameters.imgsize),
                                                        batch_size=parameters.val_batch_size,
                                                        class_mode='categorical'
                                                        ) # set as training data

classes = len(train_generator.class_indices)

Found 15674 images belonging to 15 classes.
Found 4289 images belonging to 15 classes.


In [ ]:
try:
  model = create_model(parameters.imgsize, parameters.imgsize, parameters.depth)
  train(parameters.total_epochs, model, classes)
except Exception as e:
  print(e)

71892992/71892840 [==============================] - 6s 0us/step
[LOGGING] training initial network...
Instructions for updating:
Please use Model.fit, which supports generators.
3918/3918 [==============================] - ETA: 0s - loss: 1.5741 - accuracy: 0.6042
Epoch 00001: val_accuracy improved from -inf to 0.78242, saving model to ValAccTransfer.h5
3918/3918 [==============================] - 12940s 3s/step - loss: 1.5741 - accuracy: 0.6042 - val_loss: 0.7739 - val_accuracy: 0.7824 - lr: 0.0100
[INFO] Fine tuning and continue train
[LOGGING] training final network...


In [ ]:
model_dir2 = 'ValAccFinetune.h5'

callbacks2 = [ModelCheckpoint(filepath=model_dir2, verbose=1, save_best_only=True, monitor='val_accuracy'),
                 ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1)]

model2 = load_model('FinalTransfer.h5')
model2.trainable = True
for layer in model2.layers[:-20]:
    layer.trainable = True
# initialize the optimizer compile the model and
opt = Adam(lr=parameters.FLR)
model2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])   

_ = model2.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // parameters.BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // parameters.val_batch_size,
    epochs=parameters.total_epochs,
    workers=0,
    callbacks=callbacks2,
    verbose=1)

2502/3918 [==================>...........] - ETA: 10:02 - loss: 1.4587 - accuracy: 0.5605

In [ ]:
model2.save('FinalFinetune.h5')
converter2 = tf.lite.TFLiteConverter.from_keras_model(model2)
tflite_model2 = converter2.convert()
tflite_model_name2 = "FinalFinetune.tflite"
open(tflite_model_name2, "wb").write(tflite_model2)

In [ ]:
!pip3 freeze > requirements. txt

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
# Convert the model
tflite_model = converter.convert()
# Create the tflite model file
tflite_model_name = "mymodel2.tflite"
open(tflite_model_name, "wb").write(tflite_model)

NameError: ignored

In [ ]:
# Create a converter
import tensorflow as tf
converter1 = tf.lite.TFLiteConverter.from_keras_model(model)
converter1.post_training_quantize=True

# Convert the model
tflite_model1 = converter1.convert()
# Create the tflite model file

tflite_model_name1 = "mymodel1.tflite"
open(tflite_model_name1, "wb").write(tflite_model1)

70044352

In [ ]:
from tensorflow.keras.models import load_model
model2 = load_model('fine_tuned_model.hdf5')

ValueError: ignored

In [ ]:
import cv2
import numpy as np
image = 'AF1QipO8-5PxCEmma89mG33UOc8RVrmXGfZutECKP4aas406-k-no.jpg'
image = cv2.imread(image)
image = cv2.resize(image, (128, 128))
image = image.reshape(-1, 128, 128, 3)
image = image / 255
predict = model2.predict(image)
print(predict)

[[0.00835762 0.00135387 0.0091792  0.03898181 0.07962254 0.05535886
  0.00531186 0.00157416 0.16916385 0.47641355 0.10476401 0.00421024
  0.04340877 0.00078219 0.00151738]]


In [ ]:
np.argmax(predict)

9